<a href="https://colab.research.google.com/github/prudhvijiddigam/Computational-Methods/blob/main/Jiddigam_In_class_exercise_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The ninth in-class-exercise (20 points in total, 4/16/2021)**

The purpose of the exercise is to practice different machine learning algorithms for text classification as well as the performance evaluation. In addition, you are requried to conduct *10 fold cross validation (https://scikit-learn.org/stable/modules/cross_validation.html)* in the training. 

The dataset can be download from here: https://github.com/unt-iialab/info5731_spring2021/blob/main/class_exercises/exercise09_datacollection.zip. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data. 

Algorithms:

(1) MultinominalNB

(2) SVM 

(3) KNN 

(4) Decision tree

(5) Random Forest

(6) XGBoost

Evaluation measurement:

(1) Accuracy

(2) Recall

(3) Precison 

(4) F-1 score

In [2]:
# Write your code here

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np


In [3]:
try:
    from xgboost import XGBClassifier
except ImportError ('Installing XGBoost...'):
    !pip install xgboost

In [4]:
data_train = pd.read_csv(r'/content/stsa-train.txt',sep = 'delimiter=',header= None,names=['review'])
data_test = pd.read_csv(r'/content/stsa-test.txt',sep = 'delimiter=',header= None,names=['review'])

data_train[['sentiment','review']] = data_train["review"].str.split(" ", 1, expand=True)
data_test[['sentiment','review']] = data_test["review"].str.split(" ", 1, expand=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [5]:
data_train.head()

,review,sentiment
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting-room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


In [6]:
data_test.head()

,review,sentiment
0,"no movement , no yuks , not much of anything .",0
1,"a gob of drivel so sickly sweet , even the eag...",0
2,"gangs of new york is an unapologetic mess , wh...",0
3,"we never really feel involved with the story ,...",0
4,this is one of polanski 's best films .,1


In [7]:
with open('/content/stsa-train.txt') as fin:
    data = fin.readlines()
with open('/content/stsa-test.txt') as fin:
    data.extend(fin.readlines())
sentiment = [x[0] for x in data]
review = [x.strip()[2:] for x in data]
df = pd.DataFrame({'review' : review, 'sentiment': sentiment})



In [8]:
#TEXT PROCESSING

In [13]:
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
def data_cleaning(data):
  data="".join([word.lower() for word in data if word not in string.punctuation])
  data = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", data)
  tokens = re.split('\W+',data)
  data = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return data

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(analyzer = data_cleaning)
X_tfidf = tfidf.fit_transform(data_train['review'])
print(X_tfidf.shape)

(6920, 13343)


In [16]:
X_tfidf=pd.DataFrame(X_tfidf.toarray())
X_tfidf.columns=tfidf.get_feature_names()
X_tfidf.head()

,,100minute,103minute,10course,10th,10thgrade,10year,10yearold,112minute,12,129minute,12th,12yearold,13th,14yearold,15th,15year,168minute,18yearold,1930s,1940s,1950s,1960s,1970s,1980s,19th,19thcentury,20car,20th,21st,22yearold,24andunders,26yearold,2day,30,34th,37minute,3d,3yearolds,40,...,yuen,yung,yvan,zaidan,zany,zap,zaza,zboys,zeal,zealand,zealously,zeitgeist,zelda,zellweger,zemeckis,zen,zero,zerodimensional,zeus,zhang,zhao,zhuangzhuang,zigzag,zing,zinger,zingerfilled,zip,zipper,zippy,zishe,ziyi,zoe,zombie,zombieland,zone,zoning,zoom,zwick,zzzzzzzzz,élan
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.400584,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.048154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.029784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.049596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.048782,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
test_tfidf = tfidf.transform(data_test['review'])
print(test_tfidf.shape)

(1821, 13343)


In [18]:
#splittind data into test and train data
x_train, x_test, y_train, y_test = train_test_split(X_tfidf, data_train['sentiment'].values,
                                                test_size=0.2, random_state=1)

In [19]:
naive_bayes = MultinomialNB()
svc = SVC()
knn = KNeighborsClassifier()
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()
xgb = XGBClassifier()
algos = [naive_bayes, svc, knn, dt, rf, xgb]
cut = ['nb', 'svc', 'knn', 'dt', 'rf', 'xgb']
estimators = []
for name, model in zip(cut, algos):
    cv = Pipeline([('vect', CountVectorizer()), (name,model)])
    estimators.append(cv)


In [20]:
model = naive_bayes.fit(x_train,y_train)

In [21]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

In [22]:
# Accuracy using Multinomial algorithm

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
accuracy = cross_val_score(naive_bayes, x_test, y_test, cv=18)
y_pred_naive_bayes = model.predict(x_test)
print("The accuracy for Multinomial",accuracy.mean())
print('accuracy %s' % accuracy_score(y_pred_naive_bayes,y_test))
print(classification_report(y_test,y_pred_naive_bayes))

The accuracy for Multinomial 0.7210735171261488
accuracy 0.7695086705202312
              precision    recall  f1-score   support

           0       0.81      0.69      0.74       675
           1       0.74      0.85      0.79       709

    accuracy                           0.77      1384
   macro avg       0.78      0.77      0.77      1384
weighted avg       0.77      0.77      0.77      1384



In [23]:
#Accuracy using decision tree
model1= dt.fit(x_train,y_train)



In [25]:
y_pred_decision_tree = model1.predict(x_test)
accuracy = cross_val_score(dt, x_test, y_test, cv=10)
print('accuracy' % accuracy_score(y_pred_decision_tree,y_test))
print(classification_report(y_test,y_pred_decision_tree))
print("The accuracy for decision trees",accuracy.mean())

accuracy
              precision    recall  f1-score   support

           0       0.61      0.55      0.58       675
           1       0.61      0.66      0.64       709

    accuracy                           0.61      1384
   macro avg       0.61      0.61      0.61      1384
weighted avg       0.61      0.61      0.61      1384

The accuracy for decision trees 0.5983317693671151


In [26]:
#accuracy using KNeighbours
from sklearn.model_selection import cross_val_score
accuracy = cross_val_score(knn, x_test, y_test, cv=10)
model2 = knn.fit(x_train,y_train)
y_pred_knn= model2.predict(x_test)
print('accuracy' % accuracy_score(y_pred_knn,y_test))
print(classification_report(y_test,y_pred_knn))
print("The accuracy using knn",accuracy.mean())

accuracy
              precision    recall  f1-score   support

           0       0.71      0.67      0.69       675
           1       0.70      0.74      0.72       709

    accuracy                           0.71      1384
   macro avg       0.71      0.71      0.71      1384
weighted avg       0.71      0.71      0.71      1384

The accuracy using knn 0.6633041392972578


In [28]:
from sklearn.model_selection import cross_val_score
accuracy = cross_val_score(svc, x_test, y_test, cv=10)
print("using svm",accuracy.mean())
model3 = svc.fit(x_train,y_train)
y_pred_svc = model3.predict(x_test)
print(' The accuracy using svc %s' % accuracy_score(y_pred_svc,y_test))
print(classification_report(y_test,y_pred_svc))


using svm 0.7066416432071734
 The accuracy using svc 0.7673410404624278
              precision    recall  f1-score   support

           0       0.77      0.74      0.76       675
           1       0.76      0.80      0.78       709

    accuracy                           0.77      1384
   macro avg       0.77      0.77      0.77      1384
weighted avg       0.77      0.77      0.77      1384



In [29]:
#accuracy using random forest
model4 = rf.fit(x_train,y_train)
y_pred_rf = model4.predict(x_test)
accuracy = cross_val_score(rf, x_test, y_test, cv=10)
print('The accuracy using random forest' % accuracy_score(y_pred_rf,y_test))
print(classification_report(y_test,y_pred_rf))
print("using random forest",accuracy.mean())

The accuracy using random forest
              precision    recall  f1-score   support

           0       0.73      0.61      0.67       675
           1       0.68      0.78      0.73       709

    accuracy                           0.70      1384
   macro avg       0.70      0.70      0.70      1384
weighted avg       0.70      0.70      0.70      1384

using random forest 0.6755656344489627


In [30]:
#accuracy using XG boost
model_xgb = xgb.fit(x_train,y_train)
y_pred_xgb = model_xgb.predict(x_test)
accuracy = cross_val_score(xgb, x_test, y_test, cv=10)
print('Accuracy %s' % accuracy_score(y_pred_xgb,y_test))
print(classification_report(y_test,y_pred_xgb))
print("The accuracy using XG boost",accuracy.mean())

Accuracy 0.6213872832369942
              precision    recall  f1-score   support

           0       0.72      0.37      0.49       675
           1       0.59      0.86      0.70       709

    accuracy                           0.62      1384
   macro avg       0.65      0.62      0.59      1384
weighted avg       0.65      0.62      0.60      1384

The accuracy using XG boost 0.6206808466270461


In [ ]:
From the above results we can see that Multinomial algoritham has best best accuracy and precision results.

In [38]:
#Evaluating test data

test_data_naive_bayes = model.predict(X_tfidf)
print('The accuracy on test data using Multinonial %s' % accuracy_score(test_data_naive_bayes,data_train['sentiment']))
print(classification_report(test_data_naive_bayes,data_train['sentiment']))


The accuracy on test data using Multinonial 0.9119942196531792
              precision    recall  f1-score   support

           0       0.87      0.94      0.90      3091
           1       0.95      0.89      0.92      3829

    accuracy                           0.91      6920
   macro avg       0.91      0.91      0.91      6920
weighted avg       0.91      0.91      0.91      6920

